# 最大カバー問題


In [6]:
import pulp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import japanize_matplotlib

japanize_matplotlib.japanize()
plt.style.use("ggplot")

# パラメータの設定
locations = 20  # 街灯を設置できる場所の数（これがカバーすべき地点の数と同じ）
coverage_range = 2  # 街灯がカバーできる距離
max_lamps = 5  # 設置可能な街灯の最大数
mandatory_points = [0, 5, 10, 15]  # 必ずカバーすべき地点のインデックス

# 位置情報（ランダムに生成）
np.random.seed(0)
positions = np.random.rand(locations, 2) * 10  # 共通の位置情報

# 新しい必須カバー地点の追加：駅(station)と学校(school)
station_index = 20  # 駅のインデックス
school_index = 21  # 学校のインデックス

# 駅と学校の位置を設定
station_position = np.array([1, 1])  # 駅の位置
school_position = np.array([9, 9])  # 学校の位置

# 通学路上の追加地点
intermediate_positions = np.linspace(station_position, school_position, num=5)
mandatory_points.extend([station_index, school_index] + list(range(22, 25)))

# 位置情報配列の更新
positions = np.vstack([positions, station_position, school_position, intermediate_positions[1:-1]])

# カバー行列の更新
locations = len(positions)  # 全位置の数の更新
cover = np.zeros((locations, locations), dtype=int)
for i in range(locations):
    for j in range(locations):
        if np.linalg.norm(positions[i] - positions[j]) <= coverage_range:
            cover[i, j] = 1

# PuLPの問題設定
prob = pulp.LpProblem("Street_Light_Placement", pulp.LpMaximize)

# 変数
x = pulp.LpVariable.dicts("x", range(locations), cat="Binary")
y = pulp.LpVariable.dicts("y", range(locations), cat="Binary")

# 目的関数
prob += pulp.lpSum(y[j] for j in range(locations))

# 制約条件
for j in range(locations):
    prob += y[j] <= pulp.lpSum(cover[i][j] * x[i] for i in range(locations))
prob += pulp.lpSum(x[i] for i in range(locations)) <= max_lamps

# 特定の地点を必ずカバーする制約
for j in mandatory_points:
    prob += y[j] == 1

# 求解
prob.solve()

# 結果の表示
lamp_status = [pulp.value(x[i]) for i in range(locations)]
point_covered = [pulp.value(y[j]) for j in range(locations)]

print("Status:", pulp.LpStatus[prob.status])
print("Number of lamps:", sum(lamp_status))
print("Number of covered points:", sum(point_covered))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shugo/Desktop/Lab/高大連携/code/kodai-renkei/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/l6/wzrvt4j10r97v2dkh7l274fw0000gn/T/09912522253b4467b4616e12c69b972a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/l6/wzrvt4j10r97v2dkh7l274fw0000gn/T/09912522253b4467b4616e12c69b972a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 40 COLUMNS
At line 320 RHS
At line 356 BOUNDS
At line 407 ENDATA
Problem MODEL has 35 rows, 50 columns and 154 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 19 - 0.00 seconds
Cgl0004I processed model has 22 rows, 34 columns (34 integer (32 of which binary)) and 118 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found 

In [87]:
import plotly.graph_objects as go


def create_2d_plot(
    positions: np.ndarray,
    lamp_status: list,
    point_covered: list,
    coverage_range: int,
    mandatory_points: list[int],
    station_index: int,
    school_index: int,
    route_points: list[int],  # List of indices for the commuting route
) -> go.Figure:
    is_display_legend_placed_lamps = True
    is_display_legend_not_placed_lamps = True
    is_display_legend_covered_points = True
    is_display_legend_mandatory_points = True

    fig = go.Figure()
    # Add a path for the commuting route
    route_positions = [positions[idx] for idx in [station_index, *route_points, school_index]]
    fig.add_trace(
        go.Scatter(
            x=[pos[0] for pos in route_positions],
            y=[pos[1] for pos in route_positions],
            mode="lines+markers",
            line=dict(color="black", width=4),
            marker=dict(color="black", size=10, symbol="diamond"),
            name="通学路",
        )
    )

    for i, (pos, lamp, cover) in enumerate(zip(positions, lamp_status, point_covered)):
        # Add a larger square for streetlamps
        if lamp == 1:
            fig.add_shape(
                type="circle",
                xref="x",
                yref="y",
                x0=pos[0] - 0.3,
                y0=pos[1] - 0.3,
                x1=pos[0] + 0.3,
                y1=pos[1] + 0.3,
                line_color="red",
                # fillcolor="green",
                # opacity=0.3,
                opacity=1.0,
                line_width=2,
                name="街灯位置",
                showlegend=is_display_legend_placed_lamps,
            )
            is_display_legend_placed_lamps = False
            fig.add_shape(
                type="circle",
                xref="x",
                yref="y",
                x0=pos[0] - coverage_range,
                y0=pos[1] - coverage_range,
                x1=pos[0] + coverage_range,
                y1=pos[1] + coverage_range,
                line_color="yellow",
                fillcolor="yellow",
                opacity=0.3,
                showlegend=False,
            )
        if i == station_index or i == school_index:
            # Mark station and school positions
            symbol, color, label = (
                ("x", "brown", "駅") if i == station_index else ("cross", "purple", "学校")
            )
            fig.add_trace(
                go.Scatter(
                    x=[pos[0]],
                    y=[pos[1]],
                    mode="markers",
                    marker=dict(color=color, symbol=symbol, size=15),
                    name=label,
                    showlegend=True,
                )
            )
        elif i in mandatory_points:
            # Mark mandatory points
            fig.add_trace(
                go.Scatter(
                    x=[pos[0]],
                    y=[pos[1]],
                    mode="markers",
                    marker=dict(color="red", symbol="star", size=15),
                    name="必須カバー地点",
                    showlegend=is_display_legend_mandatory_points,
                )
            )
            is_display_legend_mandatory_points = False
        elif cover == 0:
            # Mark uncovered points
            fig.add_trace(
                go.Scatter(
                    x=[pos[0]],
                    y=[pos[1]],
                    mode="markers",
                    marker=dict(color="black", symbol="circle", size=10),
                    name="カバーされていない地点",
                    showlegend=is_display_legend_covered_points,
                )
            )
            is_display_legend_covered_points = False
        else:
            # Mark covered points
            fig.add_trace(
                go.Scatter(
                    x=[pos[0]],
                    y=[pos[1]],
                    mode="markers",
                    marker=dict(color="blue", symbol="square", size=12),
                    name="カバーされた地点",
                    showlegend=is_display_legend_not_placed_lamps,
                )
            )
            is_display_legend_not_placed_lamps = False

    fig.update_layout(
        title="街灯設置の結果",
        xaxis_title="x 座標",
        yaxis_title="y 座標",
        legend_title="凡例",
        width=800,
        height=800,
        xaxis=dict(range=[-1, 12]),
        yaxis=dict(range=[-1, 12]),
    )

    return fig

In [89]:
station_index = 20
school_index = 21
route_points = [22, 23, 24]
create_2d_plot(
    positions=positions,
    lamp_status=lamp_status,
    point_covered=point_covered,
    coverage_range=coverage_range,
    mandatory_points=mandatory_points,
    station_index=station_index,
    school_index=school_index,
    route_points=route_points,
).show()